In [ ]:
!pip install -q transformers torch accelerate bitsandbytes fuzzywuzzy python-levenshtein pandas scikit-learn peft

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from fuzzywuzzy import fuzz
import time
import re
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Аугментации 

In [ ]:
categories = ['бытовая техника', 'обувь', 'одежда', 'посуда', 'текстиль', 'товары для детей', 'украшения и аксессуары', 'электроника', 'нет товара']
train_df = pd.read_csv('train_labeled.csv')  # До аугментации

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'\s+', ' ', text.strip())
    text = re.sub(r'[^\w\s.,!?—–-]', ' ', text)
    if len(text) < 10:
        return ""
    return text

train_df['text_clean'] = train_df['text_clean'].apply(preprocess_text)
train_df = train_df[train_df['text_clean'] != ''].reset_index(drop=True)
label_counts = train_df['label_final'].value_counts()
rare_classes = [cat for cat in categories if label_counts.get(cat, 0) < 100]
print(f"Редкие классы: {rare_classes}")

if rare_classes:
    model_name2 = "sberbank-ai/rugpt3large_based_on_gpt2"
    tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
    tokenizer2.pad_token = tokenizer2.eos_token
    base_model2 = AutoModelForCausalLM.from_pretrained(model_name2, torch_dtype=torch.float16).to("cuda")
    
    augmented = []
    for cat in rare_classes:
        for _ in range(150):  # 150 примеров на класс
            prompt = f"""Генерируй короткий отзыв на русском о товаре из категории '{cat}'. 
            Учитывай стиль реальных отзывов: упомяни качество, размер, доставку или недостатки.
            Пример: 'Кроссовки удобные, но быстро износились.'"""
            inputs = tokenizer2(prompt, return_tensors="pt", truncation=True, max_length=256).to("cuda")
            with torch.no_grad():
                outputs = base_model2.generate(**inputs, max_new_tokens=50, temperature=0.7, do_sample=True)
            synth_text = tokenizer2.decode(outputs[0], skip_special_tokens=True).split("Пример:")[-1].strip()
            augmented.append({'text_clean': synth_text, 'label_final': cat})
    
    aug_df = pd.DataFrame(augmented)
    train_df = pd.concat([train_df, aug_df], ignore_index=True)
    train_df.to_csv('train_labeled_augmented.csv', index=False)
    print("Аугментированный датасет сохранён как 'train_labeled_augmented.csv'")
    print("Распределение после аугментации:")
    print(train_df['label_final'].value_counts())

## Разделение датасета (train)

Хочу попробовать высчитывать weighted F1 на тренировочной выборке

In [ ]:
train_data, val_data = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df['label_final'],
    random_state=42
)
print(f"Train size: {len(train_data)}, Validation size: {len(val_data)}")
print("Validation распределение:")
print(val_data['label_final'].value_counts())

## Модель

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)

model_name = "ai-forever/mGPT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=False,
    torch_dtype=torch.float16
)
print(f"Модель загружена на {device}")

In [ ]:
def prepare_data(df, tokenizer):
    texts = [f"Отзыв: {row['text_clean']}\nКатегория: {row['label_final']}" for _, row in df.iterrows()]
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
    return encodings

train_encodings = prepare_data(train_data, tokenizer)
val_encodings = prepare_data(val_data, tokenizer)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = item['input_ids'].clone()
        return item
    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = CustomDataset(train_encodings)
val_dataset = CustomDataset(val_encodings)

## Обучение

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, lora_config)

training_args = TrainingArguments(
    output_dir="./lora_output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

start_time = time.time()
trainer.train()
model.save_pretrained("./lora_mgpt_finetuned")
tokenizer.save_pretrained("./lora_mgpt_finetuned")
print(f"Обучение завершено. Время: {time.time() - start_time:.2f}с")

## Промежуточная оценка

In [ ]:
def evaluate_model(model, tokenizer, df):
    preds = []
    prompt_template = """Ты классификатор отзывов по категориям: {categories}.
Шаги:
1. Прочитай отзыв.
2. Ищи прямые (футболка → одежда) или косвенные признаки (ткань → одежда, доставка → нет товара).
3. Выведи только категорию.

Примеры:
1. Отзыв: "Футболка велика, ткань синтетика." Категория: одежда
2. Отзыв: "Кроссовки жмут." Категория: обувь
3. Отзыв: "Заказ не пришёл." Категория: нет товара
4. Отзыв: "Ткань тонкая, швы кривые." Категория: одежда
5. Отзыв: "Телефон быстро садится." Категория: электроника

Отзыв: "{text}"
Категория:"""
    
    for text in df['text_clean']:
        prompt = prompt_template.format(categories=', '.join(categories), text=text)
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=10, temperature=0.05)
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Категория:")[-1].strip().lower()
        best_match = max(categories, key=lambda cat: fuzz.ratio(pred, cat.lower()))
        preds.append(best_match)
    
    report = classification_report(df['label_final'], preds, output_dict=True, zero_division=0)
    return report

report = evaluate_model(model, tokenizer, val_data)
print("Промежуточный Weighted F1 (validation):", report['weighted avg']['f1-score'])
print("Полный отчёт по классам:")
for label, metrics in report.items():
    if label in categories:
        print(f"{label}: Precision={metrics['precision']:.2f}, Recall={metrics['recall']:.2f}, F1={metrics['f1-score']:.2f}")

In [ ]:
test_df = pd.read_csv('test.csv')
test_df['text_clean'] = test_df['text'].apply(preprocess_text)
test_df = test_df[test_df['text_clean'] != ''].reset_index(drop=True)

report = evaluate_model(model, tokenizer, test_df)
print("Финальный Weighted F1 (test):", report['weighted avg']['f1-score'])
print("Полный отчёт по классам:")
for label, metrics in report.items():
    if label in categories:
        print(f"{label}: Precision={metrics['precision']:.2f}, Recall={metrics['recall']:.2f}, F1={metrics['f1-score']:.2f}")

In [ ]:
print("\nПервые 5 строк train (проверка корректности):")
first_five = train_df[['text_clean', 'label_final']].head(5)
for _, row in first_five.iterrows():
    text = row['text_clean']
    label = row['label_final']
    prompt = prompt_template.format(categories=', '.join(categories), text=text)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10, temperature=0.05)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Категория:")[-1].strip().lower()
    best_match = max(categories, key=lambda cat: fuzz.ratio(pred, cat.lower()))
    print(f"Отзыв: {text[:50]}...\nИстинная метка: {label}\nПредсказанная: {best_match}")